In [1]:
# Import dependencies
import numpy as np 
import numpy_financial as npf
import pandas as pd
import scipy.stats as stats
from matplotlib import pyplot as plt
from sklearn import datasets

In [2]:
# Import and clean the datasets
used_cars = pd.read_csv('Resources/Cleaned_Data.csv')
used_cars = pd.DataFrame(used_cars)
used_cars = used_cars.sort_values('Year')
used_cars = used_cars.loc[used_cars['Year'] >= 2009]
used_cars = used_cars.drop(columns = ['Unnamed: 0', 'Mileage', 'Vin'])
used_cars

,Price,Year,City,State,Make,Model
392903,9990,2009,Republic,MO,Honda,Accord
653268,5400,2009,Winston-Salem,NC,Nissan,Sentra4dr
362475,17500,2009,TULSA,OK,GMC,Yukon4WD
236724,29995,2009,Houston,TX,Dodge,Ram
658645,6995,2009,North Lauderdale,FL,Nissan,Versa5dr
...,...,...,...,...,...,...
428983,42900,2018,Fayetteville,NC,Honda,OdysseyTouring
610430,38990,2018,PEORIA,IL,Mercedes-Benz,CLACLA
428374,46090,2018,Fairfax,VA,Honda,OdysseyElite
428985,41998,2018,Winter Haven,FL,Honda,OdysseyTouring


In [39]:
inflation_table = pd.DataFrame({'Inflation Rate (%)':[17.0, 15.2, 11.6, 9.4, 7.8, 6.1, 5.9, 4.6, 2.4]})
inflation_table = (inflation_table/100) + 1
# Stack Overflow helped me add a row
inflation_table.loc[9] = [1]  # adding a row
inflation_table

,Inflation Rate (%)
0,1.170
1,1.152
2,1.116
3,1.094
4,1.078
5,1.061
6,1.059
7,1.046
8,1.024
9,1.000


In [46]:
inflation_table = pd.DataFrame({'Inflation Rate':[17.0, 15.2, 11.6, 9.4, 7.8, 6.1, 5.9, 4.6, 2.4]})
inflation_table = (inflation_table/100) + 1
# Stack Overflow helped me add a row
inflation_table.loc[9] = [1]  # adding a row
inflation_table['Year'] = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
inflation_table

,Inflation Rate,Year
0,1.170,2009
1,1.152,2010
2,1.116,2011
3,1.094,2012
4,1.078,2013
5,1.061,2014
6,1.059,2015
7,1.046,2016
8,1.024,2017
9,1.000,2018


In [47]:
used_cars_inflation = used_cars.merge(inflation_table, on = 'Year', how = 'outer')
used_cars_inflation
used_cars_inflation['Inflation Price'] = used_cars_inflation['Price'] * used_cars_inflation['Inflation Rate']
used_cars_inflation

,Price,Year,City,State,Make,Model,Inflation Rate,Inflation Price
0,9990,2009,Republic,MO,Honda,Accord,1.17,11688.30
1,5400,2009,Winston-Salem,NC,Nissan,Sentra4dr,1.17,6318.00
2,17500,2009,TULSA,OK,GMC,Yukon4WD,1.17,20475.00
3,29995,2009,Houston,TX,Dodge,Ram,1.17,35094.15
4,6995,2009,North Lauderdale,FL,Nissan,Versa5dr,1.17,8184.15
...,...,...,...,...,...,...,...,...
755350,42900,2018,Fayetteville,NC,Honda,OdysseyTouring,1.00,42900.00
755351,38990,2018,PEORIA,IL,Mercedes-Benz,CLACLA,1.00,38990.00
755352,46090,2018,Fairfax,VA,Honda,OdysseyElite,1.00,46090.00
755353,41998,2018,Winter Haven,FL,Honda,OdysseyTouring,1.00,41998.00


In [3]:
avg_fin_rate = pd.read_csv('Resources/used_avg_finrate.csv')
avg_fin_rate = pd.DataFrame(avg_fin_rate)
avg_fin_rate.head()

,DATE,RIELPCFAUNQ
0,2008-01-01,12.08
1,2008-04-01,10.63
2,2008-07-01,10.62
3,2008-10-01,10.33
4,2009-01-01,10.37


In [4]:
fin_rate = avg_fin_rate.rename(columns = {'RIELPCFAUNQ': 'Interest Rate'})
fin_rate.head()

,DATE,Interest Rate
0,2008-01-01,12.08
1,2008-04-01,10.63
2,2008-07-01,10.62
3,2008-10-01,10.33
4,2009-01-01,10.37


In [5]:
# Xpert Learning Assistant helped me figure out the str.split() method.
fin_rate[['Year', 'Month', 'Day']] = fin_rate['DATE'].str.split('-', expand = True)

In [6]:
rate_year = fin_rate.groupby('Year').mean('Interest Rate')
rate_year = rate_year.loc['2009':'2018']
rate_year = rate_year.round(2)
rate_year.index = rate_year.index.astype('int')
rate_year

,Interest Rate
Year,
2009,10.39
2010,10.89
2011,11.52
2012,12.42
2013,12.92
2014,13.33
2015,14.16
2016,13.51
2017,13.47


In [7]:
fin_amount = pd.read_csv('Resources/used_avg_finamount.csv')
fin_amount = pd.DataFrame(fin_amount)
fin_amount.head()

,DATE,DTCTLVEUANQ
0,2008-01-01,17062.62
1,2008-04-01,17095.87
2,2008-07-01,16963.08
3,2008-10-01,16947.01
4,2009-01-01,15980.77


In [8]:
fin_amount = fin_amount.rename(columns = {'DTCTLVEUANQ': 'Loan Amount'})
fin_amount[['Year', 'Month', 'Day']] = fin_amount['DATE'].str.split('-', expand = True)
fin_amount.head()

,DATE,Loan Amount,Year,Month,Day
0,2008-01-01,17062.62,2008,01,01
1,2008-04-01,17095.87,2008,04,01
2,2008-07-01,16963.08,2008,07,01
3,2008-10-01,16947.01,2008,10,01
4,2009-01-01,15980.77,2009,01,01


In [9]:
amount_year = fin_amount.groupby('Year').mean('Loan Amount')
amount_year = amount_year.loc['2009':'2018']
amount_year = amount_year.round(2)
amount_year.index = amount_year.index.astype('int')
amount_year

,Loan Amount
Year,
2009,16528.52
2010,17069.19
2011,17090.76
2012,16902.16
2013,17100.69
2014,17263.06
2015,16977.38
2016,17044.96
2017,17259.54


In [10]:
maturity = pd.read_csv('Resources/used_avg_mat.csv')
maturity = pd.DataFrame(maturity)
maturity.tail()

,DATE,DTCTLVEUMNQ
58,2022-07-01,66.19
59,2022-10-01,66.27
60,2023-01-01,65.76
61,2023-04-01,65.72
62,2023-07-01,66.38


In [11]:
maturity = maturity.rename(columns = {'DTCTLVEUMNQ': 'Months'})
maturity[['Year', 'Month', 'Day']] = maturity['DATE'].str.split('-', expand = True)
maturity_year = maturity.groupby('Year').mean('Months')/12
maturity_year = maturity_year.rename(columns = {'Months':'Loan Years'})
maturity_year = maturity_year.loc['2009':'2018']
maturity_year = maturity_year.round(2)
maturity_year.index = maturity_year.index.astype(int)
maturity_year

,Loan Years
Year,
2009,4.62
2010,4.71
2011,4.71
2012,4.74
2013,4.91
2014,4.93
2015,5.04
2016,5.08
2017,5.10


In [12]:
merge_interest = [rate_year, amount_year, maturity_year]
interest_df = pd.concat(merge_interest, axis = 1, join = 'outer')
interest_df

,Interest Rate,Loan Amount,Loan Years
Year,,,
2009,10.39,16528.52,4.62
2010,10.89,17069.19,4.71
2011,11.52,17090.76,4.71
2012,12.42,16902.16,4.74
2013,12.92,17100.69,4.91
2014,13.33,17263.06,4.93
2015,14.16,16977.38,5.04
2016,13.51,17044.96,5.08
2017,13.47,17259.54,5.10


In [13]:
used_prices = used_cars[['Price', 'Year', 'Make']]
used_prices = used_prices[(used_prices['Year'] > 2008) & (used_prices['Year'] <=2018)]
used_prices_year = used_prices.groupby('Year').mean('Price')
used_prices_year = used_prices_year.round(2)
used_prices_year

,Price
Year,
2009,13048.44
2010,14253.45
2011,16309.95
2012,17404.55
2013,19232.92
2014,22147.76
2015,25024.31
2016,25437.65
2017,29804.03


In [14]:
used_prices_merge = interest_df.merge(used_prices_year, how = 'left', on = 'Year')
# A search for calculating a loan payment formula using pandas led me to the numpy-financial pacakage and the pmt function.
used_prices_merge['Monthly Payment'] = npf.pmt((used_prices_merge['Interest Rate']/100)/12, (used_prices_merge['Loan Years'])*12, used_prices_merge['Loan Amount']).round(2)
used_prices_merge['Monthly Interest'] = (used_prices_merge['Loan Amount']) * (used_prices_merge['Interest Rate']/100) * (used_prices_merge['Loan Years'])
used_prices_merge['Monthly Interest'] = used_prices_merge['Monthly Interest'].round(2)
used_prices_merge['Total Payment'] = used_prices_merge['Loan Amount'] + used_prices_merge['Monthly Interest']
used_prices_merge

,Interest Rate,Loan Amount,Loan Years,Price,Monthly Payment,Monthly Interest,Total Payment
Year,,,,,,,
2009,10.39,16528.52,4.62,13048.44,-376.66,7933.99,24462.51
2010,10.89,17069.19,4.71,14253.45,-387.39,8755.11,25824.30
2011,11.52,17090.76,4.71,16309.95,-393.22,9273.31,26364.07
2012,12.42,16902.16,4.74,17404.55,-394.65,9950.44,26852.60
2013,12.92,17100.69,4.91,19232.92,-393.47,10848.20,27948.89
2014,13.33,17263.06,4.93,22147.76,-399.67,11344.75,28607.81
2015,14.16,16977.38,5.04,25024.31,-394.28,12116.14,29093.52
2016,13.51,17044.96,5.08,25437.65,-387.96,11698.09,28743.05
2017,13.47,17259.54,5.10,29804.03,-391.41,11856.79,29116.33


In [20]:
yearly_total_payment = used_prices_merge.drop(columns=['Price','Loan Amount', 'Monthly Payment', 'Monthly Interest'])
yearly_total_payment = yearly_total_payment.rename(columns = {'Total Payment':'Avg. Yearly Total Payment'})
yearly_total_payment

,Interest Rate,Loan Years,Avg. Yearly Total Payment
Year,,,
2009,10.39,4.62,24462.51
2010,10.89,4.71,25824.30
2011,11.52,4.71,26364.07
2012,12.42,4.74,26852.60
2013,12.92,4.91,27948.89
2014,13.33,4.93,28607.81
2015,14.16,5.04,29093.52
2016,13.51,5.08,28743.05
2017,13.47,5.10,29116.33


In [21]:
used_prices_make = used_prices.groupby(['Year', 'Make']).mean('Price')
used_prices_make = used_prices_make.round(2)
used_prices_make.reset_index(inplace = True)
used_prices_make

,Year,Make,Price
0,2009,Acura,12760.97
1,2009,Aston,70821.09
2,2009,Audi,16221.25
3,2009,BMW,14271.09
4,2009,Bentley,71503.44
...,...,...,...
408,2018,Porsche,119461.00
409,2018,Subaru,28596.50
410,2018,Toyota,24331.65
411,2018,Volkswagen,37679.53


In [22]:
used_prices_make_price = pd.merge(yearly_total_payment, used_prices_make, on = 'Year', how = 'outer')
used_prices_make_price

,Year,Interest Rate,Loan Years,Avg. Yearly Total Payment,Make,Price
0,2009,10.39,4.62,24462.51,Acura,12760.97
1,2009,10.39,4.62,24462.51,Aston,70821.09
2,2009,10.39,4.62,24462.51,Audi,16221.25
3,2009,10.39,4.62,24462.51,BMW,14271.09
4,2009,10.39,4.62,24462.51,Bentley,71503.44
...,...,...,...,...,...,...
408,2018,14.14,5.14,30868.03,Porsche,119461.00
409,2018,14.14,5.14,30868.03,Subaru,28596.50
410,2018,14.14,5.14,30868.03,Toyota,24331.65
411,2018,14.14,5.14,30868.03,Volkswagen,37679.53


In [23]:
used_prices_make_price['Monthly Payment'] = npf.pmt((used_prices_make_price['Interest Rate']/100)/12, (used_prices_make_price['Loan Years'])*12, used_prices_make_price['Price']).round(2)
used_prices_make_price['Monthly Interest'] = (used_prices_make_price['Price']) * (used_prices_make_price['Interest Rate']/100) * (used_prices_make_price['Loan Years'])
used_prices_make_price['Monthly Interest'] = used_prices_make_price['Monthly Interest'].round(2)
used_prices_make_price['Total Payment'] = used_prices_make_price['Price'] + used_prices_make_price['Monthly Interest']
used_prices_make_price

,Year,Interest Rate,Loan Years,Avg. Yearly Total Payment,Make,Price,Monthly Payment,Monthly Interest,Total Payment
0,2009,10.39,4.62,24462.51,Acura,12760.97,-290.80,6125.50,18886.47
1,2009,10.39,4.62,24462.51,Aston,70821.09,-1613.89,33995.40,104816.49
2,2009,10.39,4.62,24462.51,Audi,16221.25,-369.65,7786.49,24007.74
3,2009,10.39,4.62,24462.51,BMW,14271.09,-325.21,6850.38,21121.47
4,2009,10.39,4.62,24462.51,Bentley,71503.44,-1629.44,34322.94,105826.38
...,...,...,...,...,...,...,...,...,...
408,2018,14.14,5.14,30868.03,Porsche,119461.00,-2736.03,86823.78,206284.78
409,2018,14.14,5.14,30868.03,Subaru,28596.50,-654.95,20783.82,49380.32
410,2018,14.14,5.14,30868.03,Toyota,24331.65,-557.27,17684.15,42015.80
411,2018,14.14,5.14,30868.03,Volkswagen,37679.53,-862.98,27385.33,65064.86


In [24]:
used_prices_make_price.loc[used_prices_make_price['Total Payment'] <= used_prices_make_price['Avg. Yearly Total Payment'], 'Below Yearly Average Tot. Payment'] = 'True' 
used_prices_make_price.loc[used_prices_make_price['Total Payment'] > used_prices_make_price['Avg. Yearly Total Payment'], 'Below Yearly Average Tot. Payment'] = 'False' 
used_prices_make_price

,Year,Interest Rate,Loan Years,Avg. Yearly Total Payment,Make,Price,Monthly Payment,Monthly Interest,Total Payment,Below Yearly Average Tot. Payment
0,2009,10.39,4.62,24462.51,Acura,12760.97,-290.80,6125.50,18886.47,True
1,2009,10.39,4.62,24462.51,Aston,70821.09,-1613.89,33995.40,104816.49,False
2,2009,10.39,4.62,24462.51,Audi,16221.25,-369.65,7786.49,24007.74,True
3,2009,10.39,4.62,24462.51,BMW,14271.09,-325.21,6850.38,21121.47,True
4,2009,10.39,4.62,24462.51,Bentley,71503.44,-1629.44,34322.94,105826.38,False
...,...,...,...,...,...,...,...,...,...,...
408,2018,14.14,5.14,30868.03,Porsche,119461.00,-2736.03,86823.78,206284.78,False
409,2018,14.14,5.14,30868.03,Subaru,28596.50,-654.95,20783.82,49380.32,False
410,2018,14.14,5.14,30868.03,Toyota,24331.65,-557.27,17684.15,42015.80,False
411,2018,14.14,5.14,30868.03,Volkswagen,37679.53,-862.98,27385.33,65064.86,False
